In [2]:
import numpy as np
import pandas as pd
from pyscheduling.FS import FmCmax, FlowShop
import pickle
import time
import matplotlib.pyplot as plt
import numpy as np
import random
import numpy as np

## Makespan

In [3]:
def compute_makespan(schedule, p):
    _, m = p.shape
    n = len(schedule)
    c = [[0]*m for i in range(n)]
    for i in range(n):
        for j in range(m):
            if i == 0 and j == 0:
                c[i][j] = p[schedule[i]][j]
            elif i == 0:
                c[i][j] = c[i][j-1] + p[schedule[i]][j]
            elif j == 0:
                c[i][j] = c[i-1][j] + p[schedule[i]][j]
            else:
                c[i][j] = max(c[i][j-1], c[i-1][j]) + p[schedule[i]][j]
    return c[n-1][m-1]

In [77]:
def heuristique_NEH(M):
    n , m = M.shape
    p = M.copy()

    # Step 1: Compute the processing time of each job
    processing_time = [sum(p[i]) for i in range(n)]

    # Step 2: Sort the jobs in decreasing order of processing time
    sorted_jobs = sorted(range(n), key=lambda i: processing_time[i], reverse=True)

    # Step 3: Initialize the schedule with the first job
    schedule = [sorted_jobs[0]]

    # Step 4: Insert each subsequent job into the schedule in a position that minimizes the makespan
    for i in range(1, n):
        best_pos = -1
        best_makespan = float('inf')
        for j in range(len(schedule)+1):
            temp_schedule = schedule[:j] + [sorted_jobs[i]] + schedule[j:]
            temp_makespan = compute_makespan(temp_schedule, p)
            if temp_makespan < best_makespan:
                best_makespan = temp_makespan
                best_pos = j
        schedule.insert(best_pos, sorted_jobs[i])

    return schedule, compute_makespan(schedule, p)


def johnson_method(processing_times):
    
    jobs, machines = processing_times.shape
    #print(jobs,machines)
    copy_processing_times = processing_times.copy()
    maximum = processing_times.max() + 1
    m1 = []
    m2 = []
    
    if machines != 2:
        raise Exception("Johson method only works with two machines")
        
    for i in range(jobs):
        minimum = copy_processing_times.min()
        position = np.where(copy_processing_times == minimum)
        
        if position[1][0] == 0:
            m1.append(position[0][0])
        else:
            m2.insert(0, position[0][0])
        
        copy_processing_times[position[0][0]] = maximum
        # Delete the job appended
    return m1+m2



def heuristique_CDS(M):
    m = M.shape[1]
    solutions = []
    makespans = []

    for k in range(1,m):
        P1 = M[:,0:k].sum(axis=1) # sommer les durées sur les k premières machines
        P2 = M[:,m-k:m].sum(axis=1) # sommer les durées sur les m-k dernières machines
        P1_P2 = np.hstack([P1[:,None],P2[:,None]]) # créer une matrice à deux colonnes avec P1 et P2

        johnson_seq = johnson_method(P1_P2) # appliquer la méthode de Johnson sur P1_P2

        solutions.append(johnson_seq)
        makespans.append(np.apply_along_axis(compute_makespan, 0, johnson_seq, M)) # calculer le makespan de v avec M

    score = np.min(makespans) # trouver le score minimal
    return solutions[np.argmin(makespans)], score # renvoyer la solution correspondant au score minimal


# **Particle Swarm Optimization**

In [75]:
# initial swarm : s-1 random, 1 NEH
def init_swarm(processing_times,swarm_size,v_min,v_max):
    n_jobs=processing_times.shape[0]
    positions=[]
    velocities=[]
    best_positions=[]
    ps=[]
    i=0
    while (i<swarm_size-1):
        pos = list(range(n_jobs))
        random.shuffle(pos)
        vel = [random.uniform(v_min,v_max) for _ in range(n_jobs)]
        velocities.append(vel)
        if(pos not in positions):
            ps.append(pairwise_permute_n(pos,4))
            positions.append(pos)
            best_positions.append(pos)
            i=i+1
    pos,_ = heuristique_CDS(processing_times)
    positions.append(pos)
    best_positions.append(pos)
    ps.append(pos)
    vel = [random.uniform(v_min,v_max) for _ in range(n_jobs)]
    velocities.append(vel)
    return positions,velocities,best_positions,ps


# permutation 
def swap_perturb(schedule):
    schedule=list(schedule)
    i, j = random.sample(range(len(schedule)), 2)
    perturbed_schedule = schedule.copy()
    perturbed_schedule[i], perturbed_schedule[j] = perturbed_schedule[j], perturbed_schedule[i]
    return perturbed_schedule


def pairwise_permute_n(schedule, n_perturb):
    """
    Performs n random pairwise permutations between elements in a list.
    
    """
    new_schedule = schedule.copy()
    for i in range(n_perturb):
        idx1, idx2 = random.sample(range(len(new_schedule)), 2)
        new_schedule[idx1], new_schedule[idx2] = new_schedule[idx2], new_schedule[idx1]
    return new_schedule


# generate a new feasible sequence for a position xi
def new_seq(x_i,ps_i,y_i,g,p_i,n):
    for j in range(n):
        if y_i[j]==1 : r=ps_i[g[j]]
        elif y_i[j]==-1 : r=ps_i[p_i[j]]
        else : 
            r=-1
            i=j
            while i<n:
                if x_i[i]!=p_i[j] and x_i[i]!=g[j] : 
                    r=i
                    break 
                else : i=i+1

        rand = random.uniform(0,1)
        if (r>=j) or (rand>0.5) and (r!=-1) :
            x_i[j],x_i[r]=x_i[r],x_i[j]
            ps_i[j],ps_i[r]=ps_i[r],ps_i[j]
    return x_i,ps_i



In [71]:
def pso(processing_times,s,c1=2,c2=2,v_min=-4,v_max=4,w=0.9,alpha=0.75,max_iter=100):
    n,m = processing_times.shape
    x,v,p,ps=init_swarm(processing_times,s,v_min,v_max)
    y=[]

    #Main loop 
    for it in range (max_iter):

        #update personal best positions
        for i in range(s):
            if compute_makespan(x[i],processing_times)<compute_makespan(p[i],processing_times) :
                p[i]=x[i]
        
        #update the global best postion 
        g=p[0]
        for i in range (1,s):
            if compute_makespan(p[i],processing_times)<compute_makespan(g,processing_times) :
                g=p[i]

        for i in range(s): 
            #compute yi
            y_i=[]
            for j in range(n):
                if x[i][j]==g[j] : y_i.append(1)
                elif x[i][j]==p[i][j] : y_i.append(-1)
                elif x[i][j]==p[i][j] and x[i][j]==g[j] : 
                    rand= random.uniform(0,1)
                    if rand>0.5 : y_i.append(-1)
                    else : y_i.append(1)
                else : y_i.append(0)
            y.append(y_i)

            # update velocity
            for j in range (n):
                r1= random.uniform(0,1)
                r2=random.uniform(0,1)
                v[i][j]=v[i][j]*w+c1*r1*(-1-y[i][j])+c2*r2*(1-y[i][j])

            # update position  
            for j in range(n):
                #1. calculate l
                l=v[i][j]+y[i][j]

                #2. adjust the value of y
                if l>alpha : y[i][j]=1
                elif l<-alpha :  y[i][j]=-1
                else : y[i][j]=0

                #3. update the position 
                # if y[i][j]==1 : x[i][j]=g[j]
                # elif  y[i][j]==-1 : x[i][j]=p[i][j]
                # else : x[i][j]= random.randint(0,n-1)
            x[i],ps[i]=new_seq(x[i],ps[i],y[i],g,p[i],n)
    return g

In [78]:
instance=FmCmax.FmCmax_Instance.read_txt("../TP02-Heuristiques/data/random_instance.txt")
n = instance.n
m = instance.m
M = np.array(instance.P)
schedule=pso(M,500)
print(schedule,compute_makespan(schedule,M))

[3, 6, 5, 0, 7, 9, 1, 8, 4, 2] 1139
